# Medical Equipment Analysis

This notebook combines three key analyses:
1. **Exploratory Data Analysis (EDA)**: Overview of data quality, distributions, and categorical counts.
2. **Utilization Metrics**: Analysis of usage rates, operating hours, and procedure counts.
3. **Financial & ROI Analysis**: Calculation of revenue, costs, net profit, and ROI.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set(style="whitegrid")

# --- Load and Clean Data ---

# Get the correct file path (go up one level from analysis folder)
file_path = "../data/processed/medical_equipment_utilization_synthetic_cleaned(in).csv"

raw_df = pd.read_csv(file_path)

# Standardize column names
raw_df.columns = [c.strip().replace(" ", "_") for c in raw_df.columns]
df = raw_df.dropna(axis=1, how="all").copy()

# Clean currency columns
currency_cols = [
    "Cost_per_procedure",
    "Daily_Operating_Cost",
    "Procedure_Revenue",
    "Net-Profit_(daily)",
]
for col in currency_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("KES", "", case=False, regex=False)
            .str.replace(",", "", regex=False)
            .str.replace(" ", "", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Clean numeric columns
numeric_cols = [
    "Available_Hours",
    "Actual_Operating_Hours",
    "Utilization_Rate",
    "Number_of_Procedures",
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Convert Date
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

print("Data loaded successfully.")
print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
df.head()

## 1. Data Overview & EDA
Basic statistics, missing values, and distribution plots.

In [ ]:
print("=== Info ===")
df.info()

print("\n=== Missing Values ===")
print(df.isna().sum())

print("\n=== Descriptive Statistics ===")
print(df.describe().T)

In [ ]:
# --- Numeric Distributions ---
num_plot_cols = [
    "Available_Hours",
    "Actual_Operating_Hours",
    "Utilization_Rate",
    "Number_of_Procedures",
]

for col in num_plot_cols:
    if col in df.columns:
        plt.figure(figsize=(6, 4))
        sns.histplot(df[col], kde=True)
        plt.title(f"Distribution of {col}")
        plt.tight_layout()
        plt.show()

In [ ]:
# --- Categorical Counts ---
cat_cols = ["Equipment_Type", "Department", "Day_of_Week", "Month"]

for col in cat_cols:
    if col in df.columns:
        plt.figure(figsize=(7, 4))
        sns.countplot(data=df, x=col, order=df[col].value_counts().index)
        plt.title(f"Count of records by {col}")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

## 2. Utilization Metrics
Analyzing utilization rates, operating hours, and procedure volumes by equipment type, department, and time.

In [ ]:
# --- Aggregation by Equipment Type ---
util_by_type = (
    df.groupby("Equipment_Type")
    .agg(
        n_days=("Equipment_ID", "count"),
        avg_available_hours=("Available_Hours", "mean"),
        avg_operating_hours=("Actual_Operating_Hours", "mean"),
        avg_utilization_rate=("Utilization_Rate", "mean"),
        avg_procedures_per_day=("Number_of_Procedures", "mean"),
        total_procedures=("Number_of_Procedures", "sum"),
    )
    .sort_values("avg_utilization_rate", ascending=False)
)

print("Utilization by Equipment Type:")
display(util_by_type.round(2))

# --- Aggregation by Department ---
util_by_dept = (
    df.groupby("Department")
    .agg(
        n_days=("Equipment_ID", "count"),
        avg_utilization_rate=("Utilization_Rate", "mean"),
        total_procedures=("Number_of_Procedures", "sum"),
    )
    .sort_values("avg_utilization_rate", ascending=False)
)

print("\nUtilization by Department:")
display(util_by_dept.round(2))

In [ ]:
# --- Visualization of Utilization ---

# By Type
plt.figure(figsize=(10, 5))
sns.barplot(data=util_by_type.reset_index(), x="Equipment_Type", y="avg_utilization_rate")
plt.title("Average Utilization Rate by Equipment Type")
plt.ylabel("Utilization Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# By Department
plt.figure(figsize=(8, 5))
sns.barplot(data=util_by_dept.reset_index(), x="Department", y="avg_utilization_rate")
plt.title("Average Utilization Rate by Department")
plt.ylabel("Utilization Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# --- Monthly Trend ---
if "Month" in df.columns:
    util_by_month = (
        df.groupby("Month")
        .agg(
            avg_utilization_rate=("Utilization_Rate", "mean"),
            total_procedures=("Number_of_Procedures", "sum"),
        )
        .sort_values("avg_utilization_rate", ascending=False)
    )
    
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=util_by_month.reset_index(), x="Month", y="avg_utilization_rate", marker="o")
    plt.title("Average Utilization Rate by Month")
    plt.ylabel("Utilization Rate")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 3. Financial & ROI Analysis
Calculating Return on Investment (ROI), Profit Margins, and daily Profit/Loss.

In [ ]:
# --- ROI Metrics Calculation ---

# Group by Equipment Type
roi_by_type = (
    df.groupby("Equipment_Type")
    .agg(
        n_days=("Equipment_ID", "count"),
        total_procedures=("Number_of_Procedures", "sum"),
        total_revenue=("Procedure_Revenue", "sum"),
        total_operating_cost=("Daily_Operating_Cost", "sum"),
        total_net_profit=("Net-Profit_(daily)", "sum"),
    )
)

roi_by_type["roi_%"] = 100 * roi_by_type["total_net_profit"] / roi_by_type["total_operating_cost"]
roi_by_type["profit_margin_%"] = 100 * roi_by_type["total_net_profit"] / roi_by_type["total_revenue"]
roi_by_type["avg_profit_per_day"] = roi_by_type["total_net_profit"] / roi_by_type["n_days"]

print("Financial Metrics by Equipment Type:")
display(roi_by_type.round(2))

# Group by Department
roi_by_dept = (
    df.groupby("Department")
    .agg(
        n_days=("Equipment_ID", "count"),
        total_procedures=("Number_of_Procedures", "sum"),
        total_revenue=("Procedure_Revenue", "sum"),
        total_operating_cost=("Daily_Operating_Cost", "sum"),
        total_net_profit=("Net-Profit_(daily)", "sum"),
    )
)

roi_by_dept["roi_%"] = 100 * roi_by_dept["total_net_profit"] / roi_by_dept["total_operating_cost"]
roi_by_dept["avg_profit_per_day"] = roi_by_dept["total_net_profit"] / roi_by_dept["n_days"]

print("\nFinancial Metrics by Department:")
display(roi_by_dept.round(2))

In [ ]:
# --- Visualization of Financial Metrics ---

# ROI by Type
plt.figure(figsize=(10, 5))
sns.barplot(data=roi_by_type.reset_index(), x="Equipment_Type", y="roi_%")
plt.title("ROI (%) by Equipment Type")
plt.ylabel("ROI %")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Profit per Day by Type
plt.figure(figsize=(10, 5))
sns.barplot(data=roi_by_type.reset_index(), x="Equipment_Type", y="avg_profit_per_day")
plt.title("Average Profit per Day by Equipment Type")
plt.ylabel("Profit (KES)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()